<a href="https://colab.research.google.com/github/soodeh-nilforoushan/NLP-Transformers/blob/main/Models_%26_Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.9 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer
# import numpy as np
# import pandas as pd
# import seaborn as sn
# import matplotlib.pyplot as plt
# import textwrap

from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix

In [5]:
from tokenizers import Tokenizer
checkpoint="bert-base-uncased"
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

In [6]:
tokenizer

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [7]:
tokenizer("hello world")

{'input_ids': [101, 7592, 2088, 102], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}

In [9]:
#let's see behinde the seen of the above code:
tokens=tokenizer.tokenize("hello world")
tokens

['hello', 'world']

In [10]:
ids=tokenizer.convert_tokens_to_ids(tokens)
ids

[7592, 2088]

In [11]:
tokenizer.decode(ids)

'hello world'

In [12]:
ids=tokenizer.encode("hello world")
ids

[101, 7592, 2088, 102]

In [13]:
tokenizer.convert_ids_to_tokens(ids)

['[CLS]', 'hello', 'world', '[SEP]']

In [14]:
#bert similar to this code produce cls and sep
tokenizer.decode(ids)

'[CLS] hello world [SEP]'

In [15]:
model_inputs=tokenizer("hello world")
model_inputs

{'input_ids': [101, 7592, 2088, 102], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}

In [16]:
data=["I like cats.", "Do you like cats too?"]
tokenizer(data)

{'input_ids': [[101, 1045, 2066, 8870, 1012, 102], [101, 2079, 2017, 2066, 8870, 2205, 1029, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]}

In [17]:
from transformers import AutoModelForSequenceClassification

In [18]:
model=AutoModelForSequenceClassification.from_pretrained(checkpoint)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
#the model does not accept any kind of inputs and the defualt model for hugging face is pytorhch
#so we have to send list
outputs=model(**model_inputs)

AttributeError: ignored

In [20]:
model_inputs=tokenizer("hello world", return_tensors='pt')
model_inputs

{'input_ids': tensor([[ 101, 7592, 2088,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1]])}

In [21]:
#the deafualt was to create a binary classifier
outputs=model(**model_inputs)
outputs
#the logits are not valid because the top layer of model have not been trained

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.2083, -0.1348]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [22]:
#this time we have 3 labels for classification
model=AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
outputs=model(**model_inputs)
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-0.6471, -0.1729,  0.3495]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [26]:
outputs.logits


tensor([[-0.6471, -0.1729,  0.3495]], grad_fn=<AddmmBackward0>)

In [27]:
outputs["logits"]

tensor([[-0.6471, -0.1729,  0.3495]], grad_fn=<AddmmBackward0>)

In [28]:
outputs[0]

tensor([[-0.6471, -0.1729,  0.3495]], grad_fn=<AddmmBackward0>)

In [29]:
#if want to convert logits to numpy array it would be useful to compute accuracy
outputs.logits.detach().cpu().numpy()

array([[-0.64711535, -0.172862  ,  0.34951675]], dtype=float32)

In [30]:
data=["I like cats.", "Do you like cats too?"]
model_inputs=tokenizer(data,return_tensors='pt')
model_inputs

ValueError: ignored

In [33]:
model_inputs=tokenizer(data,padding=True,truncation=True,return_tensors='pt')
model_inputs

{'input_ids': tensor([[ 101, 1045, 2066, 8870, 1012,  102,    0,    0],
        [ 101, 2079, 2017, 2066, 8870, 2205, 1029,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1]])}

In [34]:
model_inputs['input_ids']

tensor([[ 101, 1045, 2066, 8870, 1012,  102,    0,    0],
        [ 101, 2079, 2017, 2066, 8870, 2205, 1029,  102]])

In [35]:
model_inputs["attention_mask"]

tensor([[1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1]])

In [36]:
outputs=model(**model_inputs)
outputs
#the logits us 2*3 which means we have 2 documents and we have 3 labels

SequenceClassifierOutput(loss=None, logits=tensor([[-0.7269, -0.4409,  0.5637],
        [-0.6938, -0.4716,  0.5429]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)